In [3]:
import json
import io
from configparser import ConfigParser

In [4]:
cc_json = ''

with open('./Loan_call.json', 'r') as cc_file:
    cc_json = json.loads(cc_file.read())

In [5]:
conversation = []
i=1

for phrase in cc_json['transcription']['recognizedPhrases'] :
    if i%2:
        conversation.append({"text":phrase['nBest'][0]['display'], 'id':i, 'role':'Agent', 'participantId':'Agent_{}'.format(1)})
    else:
        conversation.append({"text":phrase['nBest'][0]['display'], 'id':i, 'role':'Customer', 'participantId':'Customer_{}'.format(2)})
    i=i+1


In [6]:
json_d = { "displayName": "Conversation Task Example",
  "analysisInput": {
    "conversations": [
      {
        "conversationItems": conversation ,
        "modality": "text",
        "id": "conversation1",
        "language": "en"
      }
    ]
  },
"tasks": [   {
        "taskName": "analyze_pi_1",
        "kind": "ConversationalPIITask"
    },          
       {
        "taskName": "analyze 1",
        "kind": "ConversationalSummarizationTask",
        "parameters": {
           "summaryAspects": ["chaptertitle","issue", "resolution", "narrative"]

        }
    }
  ,             
    
  ]          
}


In [21]:
json_d

{'displayName': 'Conversation Task Example',
 'analysisInput': {'conversations': [{'conversationItems': [{'text': 'Good afternoon. This is Sam. Thank you for calling Contessa. How may I help?',
      'id': 1,
      'role': 'Agent',
      'participantId': 'Agent_1'},
     {'text': "Hi there, my name is Mary. I'm currently living in Los Angeles, but I'm planning to move to Las Vegas and I would like to apply for a loan.",
      'id': 2,
      'role': 'Customer',
      'participantId': 'Customer_2'},
     {'text': "OK, I see you're living in California. Let me make sure I understand you correctly. Uh, you'd like to apply for a loan, even though you'll be moving soon, is that right?",
      'id': 3,
      'role': 'Agent',
      'participantId': 'Agent_1'},
     {'text': "Yes, exactly. So I'm planning to relocate soon, but I would like to apply for the loan first so that I can purchase a new home once I move there.",
      'id': 4,
      'role': 'Customer',
      'participantId': 'Customer_

In [8]:
import requests
from requests.structures import CaseInsensitiveDict

parser=ConfigParser()
_=parser.read('../config.cfg')
cog_svc_ep = parser.get('openai_api','cog_svc_ep')

url = "{}/language/analyze-conversations/jobs?api-version=2022-10-01-preview".format(cog_svc_ep)

headers = CaseInsensitiveDict()
headers["Content-Type"] = "application/json"
headers["Ocp-Apim-Subscription-Key"] = parser.get('openai_api','cog_svc_key')

data = json_d

resp = requests.post(url, headers=headers, json=data)

print(resp)


<Response [202]>


In [9]:
url = resp.headers['operation-location']

In [13]:
headers = CaseInsensitiveDict()
headers["Content-Type"] = "application/json"
headers["Ocp-Apim-Subscription-Key"] = parser.get('openai_api','cog_svc_key')

data = json_d

resp = requests.get(url, headers=headers)

out = resp.json()
print(json.dumps(out['tasks'])[:200])
#print(out)
#print(json.dumps(out, indent=2))

{"completed": 2, "failed": 0, "inProgress": 0, "total": 2, "items": [{"kind": "conversationalPIIResults", "taskName": "analyze_pi_1", "lastUpdateDateTime": "2022-12-02T18:24:07.4081444Z", "status": "s


# Identifying PII

In [14]:
pii = []
for c in out['tasks']['items'][0]['results']['conversations'][0]['conversationItems']:
    if(len(c['entities'])>0) :
        print(c['redactedContent']['text'])
        pii.append(c['entities'])        
print(pii)



Good afternoon. This is ***. Thank you for calling ********. How may I help?
Hi there, my name is ****. I'm currently living in Los Angeles, but I'm planning to move to Las Vegas and I would like to apply for a loan.
Yes, my name is **********.
Thank you, Miss *****. May I have your current address please?
Yes, so my address is *********************************************************
********************. ****?
*****.
********************. Got it. Thank you. Uh, may I have your phone number as well, please?
Yes, my phone number is *04.
****************?
**.
Uh, yes. So my e-mail address is *********************.
**********. ********************************************?
****************.
*******? Uh, got it. Right. Thank you, Miss *****. Um, do you currently have any other loans?
OK, thank you. Umm, here's what I suggest we do. Uh, let me place you on a brief hold again so I can talk with one of my loan officers and get this started for you immediately. And in the meantime, uh would be

# Validate PII with OpenAI

In [15]:
import os
import openai
from configparser import ConfigParser

parser=ConfigParser()
_=parser.read('../config.cfg')
openai.api_type = "azure"
openai.api_base = parser.get('openai_api','api_ep')
openai.api_version = "2022-06-01-preview"
openai.api_key = parser.get('openai_api','api_key')
model =  parser.get('openai_api','api_model')

In [16]:
for entities in pii:
    for e in entities:
        #print(e)
        prompt = "Validate following text is type of {} with True or False:\n'''{}'''\n".format(e['category'], e['text'])
        response = openai.Completion.create( engine=model,  prompt=prompt, temperature=.1,  max_tokens=400,  top_p=0.5,  frequency_penalty=0,  presence_penalty=0,  stop=None)
        #print('Response:')
        print("{} - {}: {}".format(e['category'],e['text'],response['choices'][0]['text'].strip()))

Name - Sam: True
Name - Contessa: True
Name - Mary: True
Name - Mary Smith: True
Name - Smith: True
Address - 123 Main St in Los Angeles, CA and the ZIP code is 90923.: True
Address - Sorry, that was a 90: False
Address - What: False
Address - 90923: False
Address - 90923 on Main Street: False
Phone - 5: False
Phone - 5292351 and then: False
Phone - 51: False
Email - mary.a.sm78@gmail.com: True
Email - Mary dot a: False
Email - Was that a SN as in November or M as in Mike: False
Email - Uh, M as in Mike: False
Email - Like 78: False
Name - Smith: True
Email - www.contosoloans.com: False


# Abstractive Summary

In [16]:
for task in out['tasks']['items'][1]['results']['conversations'][0]['summaries']:
    print("{} = {}".format(task['aspect'],task['text']))


chapterTitle = Loan Application for Relocation
issue = Customer wants to apply for a loan. | Customer is planning to move to Las Vegas.
resolution = Verified the address and phone number. | Verified the e-mail address. | Verified the car loan. | Verified the student loan.
narrative = Agent_1 helps Mary Smith to apply for a loan.


In [17]:
import os
import openai
from configparser import ConfigParser

parser=ConfigParser()
_=parser.read('../config.cfg')
openai.api_type = "azure"
openai.api_base = parser.get('openai_api','api_ep')
openai.api_version = "2022-06-01-preview"
openai.api_key = parser.get('openai_api','api_key')
model =  parser.get('openai_api','api_model')

In [22]:
print("Generate detailed call summary notes of the following text:\n'''{}'''\n{}".format(conversation,'Result:\n-'))

Generate detailed call summary notes of the following text:
'''[{'text': 'Good afternoon. This is Sam. Thank you for calling Contessa. How may I help?', 'id': 1, 'role': 'Agent', 'participantId': 'Agent_1'}, {'text': "Hi there, my name is Mary. I'm currently living in Los Angeles, but I'm planning to move to Las Vegas and I would like to apply for a loan.", 'id': 2, 'role': 'Customer', 'participantId': 'Customer_2'}, {'text': "OK, I see you're living in California. Let me make sure I understand you correctly. Uh, you'd like to apply for a loan, even though you'll be moving soon, is that right?", 'id': 3, 'role': 'Agent', 'participantId': 'Agent_1'}, {'text': "Yes, exactly. So I'm planning to relocate soon, but I would like to apply for the loan first so that I can purchase a new home once I move there.", 'id': 4, 'role': 'Customer', 'participantId': 'Customer_2'}, {'text': 'And are you planning to sell your current home?', 'id': 5, 'role': 'Agent', 'participantId': 'Agent_1'}, {'text':

In [19]:
prompt = "Generate detailed call summary notes of the following text:\n'''{}'''\n{}".format(conversation,'Result:\n-')
response = openai.Completion.create( engine=model,  prompt=prompt, temperature=.1,  max_tokens=400,  top_p=0.5,  frequency_penalty=0,  presence_penalty=0,  stop=None)

print('Response:')
print(response['choices'][0]['text'])

Response:
The customer, Mary, is looking to apply for a loan in order to purchase a new home in Las Vegas
-Mary is currently living in Los Angeles but is planning to move to Las Vegas soon
-Mary is hoping to sell her current home quickly so that she can purchase a new home in Nevada and close on it quickly
-The agent takes down Mary's information including her name, address, phone number, and email address
-The agent also asks Mary if she has any other loans and Mary says she has two other loans through CONTOSO
-The agent suggests that Mary complete the remainder of the secure application online while the agent talks with one of the loan officers


In [20]:
prompt = "Generate list of follow up tasks from the following text\nText:###{}###\n".format(conversation)

response = openai.Completion.create( engine=model,  prompt=prompt, temperature=.5,  max_tokens=400,  top_p=0.5,  frequency_penalty=0,  presence_penalty=0,  stop=None)

print('Response:')
print(response['choices'][0]['text'])

Response:

-Place customer on hold to consult with loan officer
-Get started on the loan application process
-Get more information from the customer
-Complete the loan application process
